## 1. Segmentation Pre-processing

## 1.1 Dataset Conversion to fit the nnUNet
* original notebook: `BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/Dataset_conversion.ipynb`

### Preparing

In [1]:
import json
import shutil
import os 
import SimpleITK as sitk
import numpy as np

In [4]:
### 위치 정의
prefix_dir=".."
source_brats = f'{prefix_dir}/BraTS_2023_2024_solutions/GliGAN/DataSet/ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData' # BraTS2023 데이터  
source_gligan = f'{prefix_dir}/BraTS_2023_2024_solutions/GliGAN/Checkpoint/brats2023/Synthetic_dataset_random_labels' # GliGan으로 증강한 데이터
destination_folder = f'{prefix_dir}/BraTS_2023_2024_solutions/nnUNet_raw/Dataset232_BraTS_2023_rGANs' # Segmentation 데이터셋과 관련 데이터를 저장할 위치
print(os.path.exists(source_brats))
print(os.path.exists(source_gligan))
print(os.path.exists(destination_folder))

False
False
False


In [ ]:
### 환경변수 설정
os.environ['nnUNet_preprocessed'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_preprocessed"
os.environ['nnUNet_results'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_results"
os.environ['nnUNet_raw'] = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet/nnUNet_raw"

In [ ]:
### 환경변수 설정 확인
print(f"nnUNet_preprocessed: {os.environ['nnUNet_preprocessed']}")
print(f"nnUNet_results: {os.environ['nnUNet_results']}")
print(f"nnUNet_raw: {os.environ['nnUNet_raw']}")

In [ ]:
### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장하는 함수
def copy_BraTS_segmentation_and_convert_lables_to_nnUNet_2023(infile: str, out_file: str) -> None:
   img = sitk.ReadImage(in_file) # 이미지를 sitk 이미지로 변환
   img_npy = sitk.GetArrayFromImage(img) # sitk 이미지에서 numpy 정보 추출
   uniques = np.unique(img_npy) # 이미지를 구성하는 labels 확인 -> 0, 1, 2, 3 이어야함
   seg_new = np.zeros_like(img_npy) # img_npy와 동일한 크기의 0으로 채워진numpy를 만듦
   seg_new[img_npy == 3] = 3 
   seg_new[img_npy == 2] = 2
   seg_new[img_npy == 1] = 1
   seg_new[img_npy < 0] = 0
   seg_new[img_npy > 4] = 0
   img_corr = sitk.GetImageFromArray(seg_new)
   img_corr.CopyInformation(img)
   sitk.WriteImage(img_corr, out_file)

In [ ]:
### BraTS2023 데이터를 nnUNet에서 사용할 수 있게 변경하는 함수
def convert_to_nnUnet_2023_real(source_file, destination_folder):
    os.makedirs(os.path.join(destination_folder, "imagesTr"), exist_ok=True) # 'imagesTr' 폴더 작성하기
    os.makedirs(os.path.join(destination_folder, "labelsTr"), exist_ok=True) # 'labelsTr' 폴더 작성하기
    for folder in os.listdir(source_file):
        ### t1c 파일 복사히기
        t1c_name = f"{folder}-t1c.nii.gz" # 파일확인
        t1c_path = os.path.join(source_file, folder, t1c_name) # 저장할 경로 확인
        new_t1c_name = f"{folder}_0000.nii.gz" # 새 파일 이름은 뒤에 class를 적음. 0000은 0번 class.
        destination_t1c = os.path.join(destination_folder, "imagesTr", new_t1c_name) # 새 파일 경로 확인
        shutil.copy(t1c_path, destination_t1c) # 파일 복사하기

        ### t1n 파일 복사하기
        t1n_name = f"{folder}-t1n.nii.gz"
        t1n_path = os.path.join(source_file, folder, t1n_name)
        new_t1n_name = f"{folder}_0001.nii.gz"
        destination_t1n = os.path.join(destination_folder, "imagesTr", new_t1n_name)
        shutil.copy(t1n_path, destination_t1n)

        ### t2f 파일 복사하기        
        t2f_name = f"{folder}-t2f.nii.gz"
        t2f_path = os.path.join(source_file, folder, t2f_name)
        new_t2f_name = f"{folder}_0002.nii.gz"
        destination_t2f = os.path.join(destination_folder, "imagesTr", new_t2f_name)
        shutil.copy(t2f_path, destination_t2f)

        ### t2w 파일 복사하기        
        t2w_name = f"{folder}-t2w.nii.gz"
        t2w_path = os.path.join(source_file, folder, t2w_name)
        new_t2w_name = f"{folder}_0003.nii.gz"
        destination_t2w = os.path.join(destination_folder, "imagesTr", new_t2w_name)
        shutil.copy(t2w_path, destination_t2w)
        
        ### seg 파일 복사하기
        seg_name = f"{folder}-seg.nii.gz"
        seg_path = os.path.join(source_file, folder, seg_name)
        new_seg_name = f"{folder}.nii.gz"
        destination_seg = os.path.join(destination_folder, "labelTr", new_seg_name)
        
        ### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장
        copy_BraTS_segmentation_and_convert_labels_to_nnUNet_2023(in_file=seg_path, out_file=destination_seg) 

In [ ]:
### 증강하여 만든 가상의 데이터를 nnUNet에서 사용할 수 있게 변경하는 함수
def convert_to_nnUNet_2023_fake(source_file, destination_folder):
    os.makedirs(os.path.join(destination_folder, "imagesTr"), exist_ok=True)
    os.makedirs(os.path.join(destination_folder, "labelsTr"), exist_ok=True)
    for folder in os.listdir(source_file):
        ### t1ce 파일 복사히기
        t1c_name = f"{folder}-scan_t1ce.nii.gz"
        t1c_path = os.path.join(source_file, folder, t1c_name)
        new_t1c_name = f"{folder}_0000.nii.gz"
        destination_t1c = os.path.join(destination_folder, "imagesTr", new_t1c_name)
        shutil.copy(t1c_path, destination_t1c)
        
        ### t1n 파일 복사히기
        t1n_name = f"{folder}-scan_t1.nii.gz"
        t1n_path = os.path.join(source_file, folder, t1n_name)
        new_t1n_name = f"{folder}_0001.nii.gz"
        destination_t1n = os.path.join(destination_folder, "imagesTr", new_t1n_name)
        shutil.copy(t1n_path, destination_t1n)
        
        ### flair 파일 복사히기
        t2f_name = f"{folder}-scan_flair.nii.gz"
        t2f_path = os.path.join(source_file, folder, t2f_name)
        new_t2f_name = f"{folder}_0002.nii.gz"
        destination_t2f = os.path.join(destination_folder, "imagesTr", new_t2f_name)
        shutil.copy(t2f_path, destination_t2f)
        
        ### t2 파일 복사히기
        t2w_name = f"{folder}-scan_t2.nii.gz"
        t2w_path = os.path.join(source_file, folder, t2w_name)
        new_t2w_name = f"{folder}_0003.nii.gz"
        destination_t2w = os.path.join(destination_folder, "imagesTr", new_t2w_name)
        shutil.copy(t2w_path, destination_t2w)
        
        ### seg 파일 복사히기
        seg_name = f"{folder}-seg.nii.gz"
        seg_path = os.path.join(source_file, folder, seg_name)
        new_seg_name = f"{folder}.nii.gz"
        destination_seg = os.path.join(destination_folder, "labelsTr", new_seg_name)

        ### BraTS 데이터의 label을 nnUNet에서 사용할 수 있게 변경하여 저장
        copy_BraTS_segmentation_and_convert_labels_to_nnUNet_2023(in_file=seg_path, out_file=destination_seg)


### Executing

In [ ]:
### BraTS2023 데이터 변환하기
convert_to_nnUnet_2023_real(source_brats, destination_folder) # BraTS2023 데이터를 nnUNet에서 사용할 수 있게 변경

In [ ]:
### 증강하여 만든 가상의 데이터를 변환하기
convert_to_nnUnet_2023_real(source_gligan, destination_folder) # BraTS2023 데이터를 nnUNet에서 사용할 수 있게 변경

## 1.2 dataset.json for nnUNet

In [ ]:
### nnUNet에 필요한 json 파일 정보 
json_content = {
    "channel_names": {
        "0": "t1c",
        "1": "t1",
        "2": "t2f",
        "3": "t2"
    },
    "labels": {
        "background": 0,
        "whole tumor": [
            1,
            2,
            3
        ],
        "tumor core": [
            2,
            3
        ],
        "enhancing tumor": 3,
    },
    "numTraining": len(os.listdir(os.path.join(destination_folder, "labelsTr"))),
    "file_ending": ".nii.gz",
    "regions_class_order": [
        1,
        2,
        3
    ]
}

In [ ]:
### JSON 파일 저장
with open(f'{destination_folder}/dataset.json', 'w') as json_file:
    json.dump(json_content, json_file, indent=4)

## 1.3 nnUNetv2 Segmentation

In [ ]:
# Pre-processing command 만들기
### 파이썬 파일 위치 확인
nnUNetv2_plan_and_preprocess = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/experiment_planning/plan_and_preprocess_entrypoints.py"

### 파라미터 확인
segmentation_parameters = {
    "-d": [232], # 데이터셋 ID 리스트  
    "-fpe": "DatasetFingerprintExtractor", # 사용할 데이터셋 Fingerprint Extractor 클래스 이름
    "--verify_dataset_integrity": True, # 데이터셋 무결성 확인 여부
    "-np": 8, # Fingerprint 추출에 사용할 프로세스 수.
    "--clean": False, # 기존 Fingerprint를 덮어쓸지 여부
    "--verbose": False # 상세 로그 출력 여부
}

### command 생성
cmd = f'python {nnUNetv2_plan_and_preprocess}'
for key, value in segmentation_parameters.items():
        cmd += f" {key} {value}"

In [ ]:
### Pre-processing command 확인
cmd

In [ ]:
### Pre-processing command 실행
os.system(cmd)

In [ ]:
### dataset.json 파일 확인

In [ ]:
### Copy the nnUNetPlans_2023_glioma.json to the postprocessing folder
jsonfile = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/example/nnUNetPlans_2023_glioma.json"

# 2. Segmentation Training

In [ ]:
### 파이썬 위치 확인
nnUNetv2_train = f"{prefix_dir}/BraTS_2023_2024_solutions/Segmentation_Tasks/nnUNet_install/nnunetv2/run/run_training.py"